In [1]:
#In this notebook, we perform and analyze all RPE resampling
#in order to be able to compute RMS errors.
#The analogous task for GST resampling
#is performed in RMS_Error_GST.ipynb.

In [2]:
#Import relevant namespaces.
import pygsti as gst
import numpy as np
import pygsti.construction.std1Q_XY as Std1Q_XY
import time
import pickle
from pygsti.tools import rpe as RPE
from pygsti.construction import rpeconstruction as RPEConstr
from pygsti.tools.rpeconfig_GxPi2_GyPi2_UpDn import rpeconfig_GxPi2_GyPi2_UpDn

AttributeError: 'module' object has no attribute 'rpeconfig'

In [3]:
#Declare the particular RPE instance we are interested in
#(X and Y pi/2 rotations)
rpeconfig_inst = rpeconfig_GxPi2_GyPi2_UpDn

In [4]:
#Declare a variety of relevant parameters

gs_target = Std1Q_XY.gs_target
gs_target.set_all_parameterizations('TP')
maxLengths_1024 = [0,1,2,4,8,16,32,64,128,256,512,1024]
fiducials = Std1Q_XY.fiducials
#measFiducials = Std1Q_XY.fiducials
germs = gst.construction.gatestring_list([('Gy',),
 ('Gy','Gy','Gy','Gx',),
 ('Gy','Gx','Gy','Gx','Gx','Gx',),
 ('Gy','Gx','Gy','Gy','Gx','Gx',),
 ('Gy','Gy','Gy','Gx','Gy','Gx',),
 ('Gx',),
 ('Gx','Gy',),
 ('Gx','Gx','Gy','Gx','Gy','Gy',)])
stringListsGST = gst.construction.make_lsgst_lists(gs_target.gates.keys(), fiducials, fiducials, germs, maxLengths_1024)

stringListsRPE = RPEConstr.make_rpe_angle_string_list_dict(10,rpeconfig_inst)

angleList = ['alpha','epsilon','theta']

NList = [8,16,32,64,128,256]

numKs = len(maxLengths_1024[1:])
numKs_w_0 = len(maxLengths_1024)

numStrsD = {}
numStrsD['RPE'] = [6*i for i in np.arange(1,12)]
numStrsD['GST'] = [len(stringList) for stringList in stringListsGST][1:]

In [9]:
#Load the experimental datasets.
DSGST = gst.io.load_dataset('GST_dataset.txt',cache=True)
DSRPE = gst.io.load_dataset('RPE_dataset.txt',cache=True)

Loading from cache file: GST_dataset.txt.cache
Loading from cache file: RPE_dataset.txt.cache


In [7]:
#Number of times we want to resample to compute RMS error.
trials = 100

In [8]:
#Create empty dictionaries and arrays to hold various results.

gsEstD = {}
resultsD = {}
trueD = {}

method = 'RPE'
for N in NList:
    expKey = ('method',method,'N',N)
    for angle in angleList:
        resultsArray = np.zeros([numKs,trials,3],float)
        resultsD[(angle,)+expKey] = resultsArray.copy()

method = 'GST'
for angle in angleList:
    resultsArray = np.zeros([numKs,trials,3],float)
    resultsD[(angle,method,'Exp. N=370')] = resultsArray.copy()

In [10]:
#Define auxiliary function to resample from a dataset without replacement.

def resample_dataset(input_dataset,N,seed=None):
    if seed is not None:
        np.random.seed(seed)
    keys = input_dataset.keys()
    output_dataset = gst.objects.dataset.DataSet(spamLabels=input_dataset.get_spam_labels())
    for gate_string in keys:
        temp_array = [1]*input_dataset[gate_string]['plus'] + [0]*input_dataset[gate_string]['minus']
        np.random.shuffle(temp_array)
        n = np.sum(temp_array[:N])
        output_dataset.add_counts_1q(gate_string,n,N-n)
    return output_dataset

In [11]:
#Run GST on full experimental dataset; record final angle estimates.
#This should run in a minute or less.
method = 'GST'

baseKey =  ('Exp. N=370',)

DSTemp = DSGST

resultsGST = gst.do_long_sequence_gst(DSTemp,gs_target,fiducials,fiducials,germs,maxLengths_1024,verbosity=0, gaugeOptParams = {'itemWeights': {'gates':1.0, 'spam':1e-4}})

#resultsGST = Analyze.doMLEAnalysis(DSTemp,gs_target,prepFiducials,measFiducials,germs,maxLengths_1024,constrainToTP=True,advancedOptions={'verbosity':0})
gsEstD[baseKey] = resultsGST.gatesets

alphaKey = ('alpha','GST')+baseKey
epsilonKey = ('epsilon','GST')+baseKey
thetaKey = ('theta','GST')+baseKey

resultsArray = np.zeros([numKs_w_0,1,3],float)

for angle in angleList:
    key = (angle,'GST')+baseKey
    resultsD[key] = resultsArray.copy()

for kInd, k in enumerate(maxLengths_1024):
    gs = gsEstD[baseKey]['iteration estimates'][kInd]#gst.optimize_gauge(gsEstD[baseKey][kInd],'target',targetGateset=gs_target,constrainToTP=True,spamWeight=1e-4)
    resultsD[alphaKey][kInd,0,0] = RPE.extract_alpha(gs,rpeconfig_inst)
    resultsD[epsilonKey][kInd,0,0] = RPE.extract_epsilon(gs,rpeconfig_inst)
    resultsD[thetaKey][kInd,0,0] = RPE.extract_theta(gs,rpeconfig_inst)
    
for angle in angleList:
    trueD[angle] = resultsD[(angle,'GST',)+baseKey][-1,0,0]

DEBUG: optimized gate to min frobenius distance 0.000234144
DEBUG: optimized gate to min frobenius distance 0.000195541
DEBUG: optimized vector to min frobenius distance 0.000107997


In [14]:
#Encode above GST angle estimates into a "target gateset" to compare RPE results against.
gs_best_truth = RPEConstr.make_paramterized_rpe_gate_set(trueD['alpha'],trueD['epsilon'],trueD['theta'],0,rpeconfig_inst=rpeconfig_inst)

In [18]:
#Create and analyze 100 resampled datasets for several values of N.
#This should run in less than a minute.
seedCounter = 0
method = 'RPE'
startAll = time.time()
for N in NList:
    print "N =", N
    expKey = ('method',method,'N',N)
    startN = time.time()
    for index1 in xrange(trials):
        DSTemp = resample_dataset(DSRPE,N,seed=seedCounter)
        resultsD[('seedCounter-index1',index1)+expKey] = seedCounter
        seedCounter += 1
        resultsTemp = RPE.analyze_rpe_data(DSTemp,gs_best_truth,stringListsRPE,rpeconfig_inst)
        for angle in angleList:
            resultsD[(angle,)+expKey][:,index1:index1+1,0] = np.array([resultsTemp[angle+'ErrorList']]).T.copy()
        del resultsTemp
        del DSTemp
    endN = time.time()
    print endN - startN
endAll = time.time()
print endAll - startAll

N = 8
6.71701598167
N = 16
6.68987989426
N = 32
6.84075808525
N = 64
6.9906771183
N = 128
6.42323279381
N = 256
6.71462702751
40.3788089752


/Users/kmrudin/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/IPython/kernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [17]:
#Save above results, stored in the dictionary resultsD, as a pickle.
RPEResultsFile = open('RPE-Subsampling-Results.pkl','w')
pickle.dump(resultsD,RPEResultsFile)
RPEResultsFile.close()

In [20]:
resultsD.keys()

[('seedCounter-index1', 81, 'method', 'RPE', 'N', 64),
 ('seedCounter-index1', 9, 'method', 'RPE', 'N', 256),
 ('seedCounter-index1', 6, 'method', 'RPE', 'N', 256),
 ('seedCounter-index1', 13, 'method', 'RPE', 'N', 64),
 ('seedCounter-index1', 58, 'method', 'RPE', 'N', 32),
 ('seedCounter-index1', 65, 'method', 'RPE', 'N', 32),
 ('seedCounter-index1', 36, 'method', 'RPE', 'N', 16),
 ('seedCounter-index1', 22, 'method', 'RPE', 'N', 64),
 ('seedCounter-index1', 68, 'method', 'RPE', 'N', 64),
 ('seedCounter-index1', 35, 'method', 'RPE', 'N', 8),
 ('seedCounter-index1', 93, 'method', 'RPE', 'N', 8),
 ('seedCounter-index1', 63, 'method', 'RPE', 'N', 8),
 ('seedCounter-index1', 4, 'method', 'RPE', 'N', 16),
 ('seedCounter-index1', 19, 'method', 'RPE', 'N', 16),
 ('seedCounter-index1', 94, 'method', 'RPE', 'N', 8),
 ('seedCounter-index1', 61, 'method', 'RPE', 'N', 64),
 ('seedCounter-index1', 66, 'method', 'RPE', 'N', 8),
 ('epsilon', 'method', 'RPE', 'N', 128),
 ('seedCounter-index1', 6, 'me